# 2.2.3 節
コメントは、勉強会で扱った後に充実させる。

### 準備

リスト用ユーティリティ+$\alpha$

In [1]:
(define nil (list))
(define (push x xs)
  (if (null? xs) (list x)
      (cons (car xs) (push x (cdr xs)))))
(define (leaf? t)
  (not (pair? t)))

(define (show-lines . xs)
  (define (iter xs)
    (cond ((null? xs) #t)
          (else (display (car xs))
                (newline)
                (iter (cdr xs)))))
  (iter xs))
(define (square x) (* x x))

以下は、この節の本文中で定義されたもの。

`enumerate-tree`は私の教義に従って定義を変えた。

In [2]:
(define (filter predicate seq)
  (cond ((null? seq) nil)
        ((predicate (car seq))
         (cons (car seq) (filter predicate (cdr seq))))
        (else (filter predicate (cdr seq)))))
(define (accumulate op init seq)
  (if (null? seq) init
      (op (car seq) (accumulate op init (cdr seq)))))
(define (enumerate-interval low high)
  (if (> low high) nil
      (cons low (enumerate-interval (+ low 1) high))))
(define (enumerate-tree tree)
  (if (leaf? tree) (list tree)
      (accumulate append nil
                  (map enumerate-tree tree))))

これらを利用して、後で役立つ手続きを定義しておく。

※`accumulate`が右結合なので`and`や`or`の短絡性を活かしきれていない。
辛うじて`y`を先に評価することで、真理値が確定してからはより左の`(predicate x)`は評価しないようにしているが、
走査はしてしまう。

In [3]:
(define (all predicate seq)
  (accumulate (lambda (x y) (and y (predicate x))) #t seq))
(define (any predicate seq)
  (accumulate (lambda (x y) (or y (predicate x))) #f seq))
(define (take-while predicate seq)
  (if (or (null? seq) (not (predicate (car seq)))) []
           (cons (car seq) (take-while predicate (cdr seq)))))

(define (prime? n)
  (define (prime-to-n? m)
    (not (= (remainder n m) 0)))
  (and (> n 1)
       (all prime-to-n?
            (take-while (lambda (x) (<= (* x x) n))
                        (enumerate-interval 2 (- n 1))))))
(prime? 134)

#f

## Ex. 2.33
第一引数にどのような手続き`f`を与えるかが本質。
この`f`は、
1. リストの新しい要素（`x`）と
2. そこまでの集計結果（`y`）を受け取り、
3. 新しい集計結果を返す。

たとえば、`map`について考えると、
1. `x`は`p`を適用される前の新しい要素、
2. `y`は`map`済みの部分リストであるので、
3. `(p x)`を`y`の先頭に追加すれば、新たな集計結果が得られる。

したがって、`f` := `(lambda (x y) (cons (p x) y))`。
以下同様。

再帰的定義と同様に、`y`をすでに計算されたものとして捉えることがポイント。

なお、ここで`map`を上書きしてしまうとEx. 2.37に支障が出るので名前を変えてある。

In [4]:
(define (map2 p sequence)
  (accumulate (lambda (x y) (cons (p x) y)) nil sequence))
(define (append seq1 seq2)
  (accumulate cons seq2 seq1))
(define (length sequence)
  (accumulate (lambda (_ y) (+ 1 y)) 0 sequence))

## Ex. 2.34
前問と同様だが、途中結果`y`の意味を想像しにくいときは定義に戻るのも手である。Ex. 2.38も参照。

In [5]:
(define (horner-eval x coefficient-sequence)
  (accumulate (lambda (this-coeff higher-terms) (+ this-coeff (* higher-terms x)))
              0
              coefficient-sequence))

(horner-eval 2 (list 1 3 0 5 0 1))

79

## Ex. 2.35
与えられたコードは次の通り。

    (define (count-leaves t)
      (accumulate (?1) (?2) (map (?3) (?4))))
 
穴が多すぎて理詰めで埋めるのは大変であるが、
「部分ツリーの葉数をカウントして結果を合計する」というスケッチに至れば、
`(?1)`、`(?2)`、`(?4)`は直ちに埋まる。

* `(?1)`: `+`
* `(?2)`: `0`
* `(?4)`: `t`

`(?3)`は部分ツリーの葉数を求める手続きということで、
詰まるところ`count-leaves`を再帰すればいいのであるが、
ベースケース（葉の場合）もここで処理する必要があるのでもう少し煩雑になる。

In [6]:
(define (count-leaves t)
  (accumulate +
              0
              (map (lambda (sub-tree) (if (leaf? sub-tree) 1 (count-leaves sub-tree))) t)))

(count-leaves (list (list 1 (list 2 3) 4) (list 5 6 (list 7 8 9))))

9

### 補註
以上が解答であるが、再帰を用いるのであれば、
何度も言うように、ベースケースは手続きの最初で処理するのが鉄則である。
記述がシンプルになるだけでなく、`t`が「葉のみ」の場合にも対応できる。
（このケースもツリーに含める方が、データ構造の定義はシンプルになる。）

In [7]:
(define (count-leaves t)
  (if (leaf? t) 1
      (accumulate +
                  0
                  (map count-leaves t))))
(count-leaves 4)

1

また、本文中で定義した`enumerate-tree`（i.e. `fringe`）を用いれば、再帰に頼らずとも定義できる。
これは別解である。

In [8]:
(define (count-leaves t)
  (accumulate (lambda (_ y) (+ 1 y))
              0
              (enumerate-tree t)))

(count-leaves (list (list 1 (list 2 3) 4) (list 5 6 (list 7 8 9))))

9

ただ、それをするのであれば、Ex. 2.33で定義した`length`を用いてこう書く。

In [9]:
(define (count-leaves t) (length (enumerate-tree t)))

## Ex. 2.36
「リストのリスト」を縦断するので、ここまでのように手続きを直列する訳にはいかず、
基本に戻って丁寧に再帰していく。

In [10]:
(define (accumulate-n op init seqs)
  (if (null? (car seqs)) nil
      (cons (accumulate op init (map car seqs))
            (accumulate-n op init (map cdr seqs)))))

(accumulate-n + 0 (list (list 1 2 3) (list 4 5 6) (list 7 8 9) (list 10 11 12)))

(22 26 30)

### 補註
`accumulate-n`では、`seqs`の要素である各リストから先頭要素だけを切り出しながら、`accumulate`で集約している。
前半の「行を切り出す」手続き`transpose`を部品化しておけば、直列的に定義できる。

In [11]:
(define (transpose seqs)
  (if (null? (car seqs)) nil
      (cons (map car seqs)
            (transpose (map cdr seqs)))))
(define (accumulate-n2 op init seqs)
  (map (lambda (seq) (accumulate op init seq))
       (transpose seqs)))

(accumulate-n2 + 0 (list (list 1 2 3) (list 4 5 6) (list 7 8 9) (list 10 11 12)))

(22 26 30)

逆に、`accumulate-n`から`transpose`を定義することも可能。Ex. 2.37参照。
`accumulate-n`の名前を変えたのは、その際に無限の相互再帰に陥るのを防ぐため。

## Ex. 2.37

In [12]:
(define (dot-product v w)
  (accumulate + 0 (map * v w)))

(dot-product (list 1 2) (list 2 3))

8

「行列 = 行ベクトルのリスト」ということを意識すれば、リスト処理の範疇に収まる。

$mv$はベクトルであり、その各要素は`m`の行ベクトルと`v`の内積である。

In [13]:
(define (matrix-*-vector m v)
  (map (lambda (row) (dot-product row v)) m))

`transpose`は行列の各行を縦断しながら、縦に`cons`する。

In [14]:
(define (transpose mat)
  (accumulate-n cons nil mat))

(transpose
 (list (list 1 2 3)
       (list 4 5 6)))

((1 4) (2 5) (3 6))

$\verb+m+ \verb+n+$は行列であり、その各行ベクトルは、`m`の行ベクトルに`n`を**右から**作用させたものである。

$$
mn = \begin{pmatrix} \vdots \\ m_i \\ \vdots \end{pmatrix} n
= \begin{pmatrix} \vdots \\ m_i n \\ \vdots \end{pmatrix}
$$

ベクトルに行列を右から作用させる`vector-*-matrix`は未定義だが、
これは（行ベクトルと列ベクトルの違いを除けば）行列の転置を左から作用させることに他ならない。

$$
(m_i n)^t = n^t m_i^t
$$

In [15]:
(define (matrix-*-matrix m n)
  (let ((cols (transpose n)))
    (map (lambda (row) (matrix-*-vector cols row)) m)))

(matrix-*-matrix
 (list (list 1 2)
       (list 3 4))
 (list (list 1 0)
       (list 0 0)))

((1 0) (3 0))

### 補註
冗長ながら、最後の定義を改めて代数的に説明する。
$m_i$、$n_i$はそれぞれ$m$、$n$の$i$行ベクトルである。
$mn$の転置について考えると、

$$
(m n)^t
= n^t m^t
= n^t \begin{pmatrix} \dots m_i^t \dots \end{pmatrix}
= \begin{pmatrix}\dots n^t m_i^t \dots \end{pmatrix}
$$

ここから、$(mn)^t$の列ベクトル、
すなわち、$mn$の行ベクトルが、$n^t$を$m$の行ベクトルに作用させたものであることがわかる。


## Ex. 2.38
動作イメージ。
$$\verb+(fold-right * i a)+
= \verb+a+_1 * ( \dots * (\verb+a+_{n-1} * (\verb+a+_n * i)))$$
$$\verb+(fold-left * i a)+
= (((i*\verb+a+_1) * \verb+a+_2) * \dots ) * \verb+a+_n$$

In [16]:
(define (fold-right op init seq) (accumulate op init seq))
(define (fold-left op init seq)
  (define (iter result rest)
    (if (null? rest) result
        (iter (op result (car rest)) (cdr rest))))
  (iter init seq))

$$
1 / (2 / (3 / 1)) = 3/2
$$
$$
((1 / 1) / 2) / 3 = 1/6
$$
後半はそれぞれの作用を可視化するためのもので、前述のイメージそのもの。

In [17]:
(show-lines
 (fold-right / 1 (list 1 2 3))
 (fold-left / 1 (list 1 2 3))
 (fold-right list nil (list 1 2 3))
 (fold-left list nil (list 1 2 3)))

3/2
1/6
(1 (2 (3 ())))
(((() 1) 2) 3)


#t

「あらゆる列に対して」と言ったときに、`init`について考えないのであれば、
`op`が**結合的**であることが、2つの手続きが同じ結果を生み出すために必要十分。
$$\verb+(op (op x y) z)+ = \verb+(op x (op y z))+$$

`init`についても考慮する場合、
両者では`init`が異なる向きから演算されるので、
それによって結果が異なることがありうる。
`init`が`op`の単位元であれば十分であるが、
それ以外の場合でも例外的に結果が一致することはありうるので、必要十分条件を挙げることは難しい。

## Ex. 2.39
`fold-right`（i.e. `accumulate`）についてはEx. 2.33で詳しく述べた。
同様に、`fold-left`について考えると、第一引数`f`は、
1. そこまでの集計結果（`x`）と
2. リストの新しい要素（`y`）を受け取り、
3. 新しい集計結果を返す。

と、`x`と`y`の役割が逆転するので注意。

In [18]:
(define (reverse seq)
  (fold-right (lambda (x y) (push x y)) nil seq))
(reverse (list 1 2 3 4))

(4 3 2 1)

In [19]:
(define (reverse seq)
  (fold-left (lambda (x y) (cons y x)) nil seq))
(reverse (list 1 2 3 4))

(4 3 2 1)

### 補註
オーダーについて。
$n$を`seq`の長さとして、
`push`のステップ数は$\Theta(n)$、`cons`は$\Theta(1)$であるので、
新しく定義される`reverse`は、それぞれ$\Theta(n^2)$と$\Theta(n)$のオーダーを持つ。

`reverse`のオーダーを縮められることはEx. 2.22でも述べたが、
注意深く観察すれば、
fold-leftを用いて定義される`reverse`は、反復プロセスであることを含めて、
そこで定義したものと全く同じプロセスを生成することがわかるだろう。

---

In [20]:
(define (flatmap proc seq)
  (accumulate append nil (map proc seq)))

## Ex. 2.40
本文p.167にほとんど答えが書いてある。`accumulate`で直接記述されていた部分を`flatmap`で書き換えるくらいである。

In [21]:
(define (unique-pairs n)
  (flatmap (lambda (i)
             (map (lambda (j) (list i j)) (enumerate-interval 1 (- i 1))))
           (enumerate-interval 1 n)))

(unique-pairs 4)

((2 1) (3 1) (3 2) (4 1) (4 2) (4 3))

In [22]:
(define (prime-sum? pair)
  (prime? (+ (car pair) (cadr pair))))
(define (make-pair-sum pair)
  (list (car pair) (cadr pair) (+ (car pair) (cadr pair))))
(define (prime-sum-pairs n)
  (map make-pair-sum
       (filter prime-sum?
               (unique-pairs n))))

(prime-sum-pairs 10)

((2 1 3) (3 2 5) (4 1 5) (4 3 7) (5 2 7) (6 1 7) (6 5 11) (7 4 11) (7 6 13) (8 3 11) (8 5 13) (9 2 11) (9 4 13) (9 8 17) (10 1 11) (10 3 13) (10 7 17) (10 9 19))

### 補註
だが、せっかくなので気持ちのよい順番（辞書順）に並ぶように改良しておく。

In [23]:
(define (unique-pairs n)
  (flatmap (lambda (i)
             (map (lambda (j) (list i j)) (enumerate-interval (+ i 1) n)))
           (enumerate-interval 1 n)))

(unique-pairs 4)

((1 2) (1 3) (1 4) (2 3) (2 4) (3 4))

In [24]:
(prime-sum-pairs 10)

((1 2 3) (1 4 5) (1 6 7) (1 10 11) (2 3 5) (2 5 7) (2 9 11) (3 4 7) (3 8 11) (3 10 13) (4 7 11) (4 9 13) (5 6 11) (5 8 13) (6 7 13) (7 10 17) (8 9 17) (9 10 19))

## Ex. 2.41
改良版`unique-pair`を想定した設計。
`cadr`を`car`に変えれば元の版にも一応対応できる。

In [25]:
(define (unique-triples n)
  (flatmap (lambda (pair)
             (map (lambda (k) (push k pair))
                  (enumerate-interval (+ (cadr pair) 1) n)))
           (unique-pairs n)))

(unique-triples 6)

((1 2 3) (1 2 4) (1 2 5) (1 2 6) (1 3 4) (1 3 5) (1 3 6) (1 4 5) (1 4 6) (1 5 6) (2 3 4) (2 3 5) (2 3 6) (2 4 5) (2 4 6) (2 5 6) (3 4 5) (3 4 6) (3 5 6) (4 5 6))

In [26]:
(define (s-sum-triples s n)
  (define (s-sum? triple)
    (= (+ (car triple) (cadr triple) (caddr triple)) s))
  (filter s-sum? (unique-triples n)))

(s-sum-triples 20 10)

((1 9 10) (2 8 10) (3 7 10) (3 8 9) (4 6 10) (4 7 9) (5 6 9) (5 7 8))

## Ex. 2.42
$(x_1, y_1)$と$(x_2, y_2)$が、
* 同じ列上にある $\Leftrightarrow$ $x_1 = x_2$
* 同じ行上にある $\Leftrightarrow$ $y_1 = y_2$
* 同じ順対角線（左上-右下）上にある $\Leftrightarrow$ $x_1 + y_2 = x_2 + y_1$
* 同じ逆対角線（左下-右上）上にある $\Leftrightarrow$ $x_1 + y_1 = x_2 + y_2$


In [27]:
(define (queens board-size)
  (define (queen-cols k)
    (if (= k 0) (list empty-board)
        (filter safe?
         (flatmap
          (lambda (rest-of-queens)
            (map (lambda (new-row)
                   (adjoin-position
                    new-row k rest-of-queens))
                 (enumerate-interval 1 board-size)))
          (queen-cols (- k 1))))))
  (queen-cols board-size))

(define empty-board nil)
(define (safe? positions)
  (let ((new-pos (car positions)) (rest-pos (cdr positions)))
    (define (safe-to-new-pos? pos2)
      (let ((x1 (car new-pos)) (y1 (cadr new-pos)) (x2 (car pos2)) (y2 (cadr pos2)))
        (not (or (= x1 x2)
                 (= y1 y2)
                 (= (+ x1 y1) (+ x2 y2))
                 (= (+ x1 y2) (+ x2 y1))))))
    (all safe-to-new-pos? rest-pos)))

(define (adjoin-position new-row k rest-of-positions)
  (cons (list new-row k) rest-of-positions))

(queens 5)

(((4 5) (2 4) (5 3) (3 2) (1 1)) ((3 5) (5 4) (2 3) (4 2) (1 1)) ((5 5) (3 4) (1 3) (4 2) (2 1)) ((4 5) (1 4) (3 3) (5 2) (2 1)) ((5 5) (2 4) (4 3) (1 2) (3 1)) ((1 5) (4 4) (2 3) (5 2) (3 1)) ((2 5) (5 4) (3 3) (1 2) (4 1)) ((1 5) (3 4) (5 3) (2 2) (4 1)) ((3 5) (1 4) (4 3) (2 2) (5 1)) ((2 5) (4 4) (1 3) (3 2) (5 1)))

## Ex. 2.43

このコードを実行してはいけない。

In [28]:
(define (queens board-size)
  (define (queen-cols k)
    (if (= k 0) (list empty-board)
        (filter safe?
         (flatmap
          (lambda (new-row)
            (map (lambda (rest-of-queens)
                   (adjoin-position
                    new-row k rest-of-queens))
                 (queen-cols (- k 1))))
          (enumerate-interval 1 board-size)))))
  (queen-cols board-size))

(define empty-board nil)
(define (safe? positions)
  (let ((new-pos (car positions)) (rest-pos (cdr positions)))
    (define (safe-to-new-pos? pos2)
      (let ((x1 (car new-pos)) (y1 (cadr new-pos)) (x2 (car pos2)) (y2 (cadr pos2)))
        (not (or (= x1 x2)
                 (= y1 y2)
                 (= (+ x1 y1) (+ x2 y2))
                 (= (- x1 y1) (- x2 y2))))))
    (all safe-to-new-pos? rest-pos)))

(define (adjoin-position new-row k rest-of-positions)
  (cons (list new-row k) rest-of-positions))

;(queens 5)

ポイントは、`(queen-cols k)`の呼び出し1回に対して、`(queen-cols (- k 1))`が呼び出される回数。

元のヴァージョンでは、この記述は、手続き本体のサブフォーミュラ（部分形式）として記述されていたため、
その評価は1度だけであった。

今回のコードでは、`(queen-cols (- k 1))`はラムダ式の中に記述されている。
すでに扱ったように、ラムダ式の中身が評価されるのは、それが引数に対して適用されたタイミングである。
このラムダ式は`map`の引数であるので、`(enumerate-interval 1 board-size)`のそれぞれに対して適用される。
すなわち、**`(queen-cols k)`の呼び出し1回に対して`(queen-cols (- k 1))`が`board-size`回呼び出される**。

見積もりは後に回す。